# Visualisatiot of scors in map

## EIT results

In [133]:
!pip install altair

import altair as alt
import pandas as pd
import file_processing_EIT

alt.data_transformers.enable('default')

DataTransformerRegistry.enable('default')

Group by EIT preparation region, and sort by descending average score.

In [134]:
eit = file_processing_EIT.EIT

eit_by_region = eit.groupby('EORegName')['Average Score'].mean().reset_index()
eit_by_region = eit_by_region.sort_values('Average Score', ascending=False)

We translate the names of the regions into English according to the content of the JSON file describing the map of Ukraine.

In [135]:
mapping = {
    "Черкаська область": "Cherkasy", "Чернігівська область ": "Chernihiv", "Чернівецька область": "Chernivtsi",
    "Дніпропетровська область": "Dnipropetrovs'k", "Донецька область": "Donetsk", "Івано-Франківська область": "Ivano-Frankivs'k",
    "Харківська область": "Kharkiv", "Херсонська область": "Kherson", "Хмельницька область": "Khmel'nyts'kyy",
    "Київська область": "Kiev", "Кіровоградська область": "Kirovohrad", "Луганська область": "Luhansk",
    "Львівська область": "L'viv", "Миколаївська область": "Mykolayiv", "Одеська область": "Odessa",
    "Полтавська область": "Poltava", "Рівненська область": "Rivne", "Сумська область": "Sumy",
    "Тернопільська область": "Ternopil'", "Вінницька область": "Vinnytsya", "Волинська область": "Volyn",
    "Закарпатська область": "Transcarpathia", "Запорізька область": "Zaporizhzhya", "Житомирська область": "Zhytomyr",
    "м.Київ" : "Kiev City"
}

eit_by_region['EORegName'] = eit_by_region['EORegName'].map(mapping)

Some areas are not in the dataset itself. Therefore, we add them manually and assign a score of 0. (so that they are displayed on the map)

In [136]:
missing_regions = ['Chernihiv', "Luhans'k", "Donets'k", 'Crimea']

missing_data = pd.DataFrame({
    'EORegName': missing_regions,
    'Average Score': 0
})

eit_by_region = pd.concat([eit_by_region, missing_data], ignore_index=True)

In the final, we display the results

In [137]:
ukraine = alt.topo_feature(
    "https://raw.githubusercontent.com/org-scn-design-studio-community/sdkcommunitymaps/master/geojson/Europe/Ukraine-regions.json",
    'UKR_adm1',)

mapa = alt.Chart(ukraine).mark_geoshape().encode(
    color='Average Score:Q'
).transform_lookup(
    lookup='properties.NAME_1',
    from_=alt.LookupData(eit_by_region, 'EORegName', ['Average Score'])
).properties(
    width=800,
    height=400
)

mapa

alt.Chart(...)

In [138]:
eit_by_region

,EORegName,Average Score
0,Kiev City,145.534003
1,L'viv,143.142194
2,Kharkiv,140.788680
3,Volyn,139.062445
4,Kiev,138.889807
5,Cherkasy,138.787695
6,Sumy,138.600807
7,NaN,138.058044
8,Ternopil',137.833239
9,Khmel'nyts'kyy,137.595582


## NMT results